In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from ipywidgets import interact
import ipywidgets as widgets
import folium
import warnings
warnings.simplefilter('ignore')
from IPython.display import display, HTML
import re

In [2]:
 %%HTML
<p><img src="banner1.png" align="center"></img></p>

In [67]:
%%HTML

<div style = "text-align: justify;">
<div style = "Lucida Console, Courier, monospace;font-size:20px;color:#778899;text-align:center;" ><h2>Interactive Analysis Dashboard</h2></div>
<div style = "Lucida Console, Courier, monospace;font-size:20px;color:#778899;line-height: 1.6;text-align: justify;" >Coronaviruses are a large family of viruses that may cause respiratory illnesses in humans ranging from common colds to more severe conditions such as Severe Acute Respiratory Syndrome (SARS) and Middle Eastern Respiratory Syndrome (MERS). Novel coronavirus is a new, previously unidentified strain of coronavirus. The coronavirus involved in the current outbreak has been named SARS-CoV by the World Health Organization (WHO).The disease it causes has been named “coronavirus disease 2019” or “COVID-19”.</div>
</div>'

In [43]:
# Loading the online data from csv files 

country_df = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/web-data/data/cases_country.csv')
death_df = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv')
confirmed_df = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv')
recovered_df = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv')


In [44]:
#country_df.head() # recovered_df['country']   # country_df.head(),death_df.head(),confirmed_df.head()

In [45]:
# coverting all the coloumss into lower cases using map function 

country_df.columns = map(str.lower, country_df.columns)
confirmed_df.columns = map(str.lower, confirmed_df.columns)
death_df.columns = map(str.lower, death_df.columns)
recovered_df.columns = map(str.lower, recovered_df.columns)

In [46]:
# changing province/state to state and country/region to country

confirmed_df = confirmed_df.rename(columns={'province/state': 'state', 'country/region': 'country'})
recovered_df = confirmed_df.rename(columns={'province/state': 'state', 'country/region': 'country'})
death_df = death_df.rename(columns={'province/state': 'state', 'country/region': 'country'})
country_df = country_df.rename(columns={'country_region': 'country'})
#country_df.head()

In [49]:
last_update = country_df['last_update'].values[0]
display(HTML('<div style = "font-family:Trebuchet MS;font-size:15px;color:#778899;"><h1>Last Updated</h1></div>'))

def change_date_format(dt):
        return re.sub(r'(\d{4})-(\d{1,2})-(\d{1,2})', '\\3-\\2-\\1', dt)
dt1 = "2026-01-02"
print("\033[1;34;143m", change_date_format(last_update), "\n")


 16-05-2020 06:33:17 



In [60]:



fig = go.FigureWidget( layout=go.Layout() )

def highlight_col(x):
    
                    country   = 'background-color: #ffffcc'
                    confirmed = 'background-color: #99ebff'
                    death     = 'background-color: #ffc2b3'
                    recovered = 'background-color: #99ff99'
                    active    = 'background-color: #ffb3ff'
                    

                    df1 = pd.DataFrame('', index=x.index, columns=x.columns)
                    df1.iloc[:, 0] = country
                    df1.iloc[:, 1] = confirmed
                    df1.iloc[:, 2] = death
                    df1.iloc[:, 3] = recovered
                    df1.iloc[:, 4] = active
                   


                    return df1

def show_latest_cases(n):
    n = int(n)
    if 'lat' in country_df:
        country_df.drop(['lat','long_','last_update','incident_rate','people_tested','people_hospitalized','mortality_rate','uid','iso3'], axis =1, inplace =True)

    s = country_df.sort_values('confirmed', ascending= False).head(n).style.apply(highlight_col, axis=None) 
    s = s.hide_index()
    #s = s.set_caption('<h3>Top Worst Hit Countries</h3><br>')
    return s;

display(HTML('<br><br><div style = "font-family:Trebuchet MS;font-size:30px;color:#778899;">Enter the Number to View World Top Worst Hit Countries</div><br>'))
interact(show_latest_cases, n='10');


interactive(children=(Text(value='10', description='n'), Output()), _dom_classes=('widget-interact',))

In [40]:
sorted_country_df = country_df.sort_values('confirmed',ascending=False)

In [41]:
confirmed_ref_val_curr =        confirmed_df.iloc[:,-1]
confirmed_ref_val_previous =    confirmed_df.iloc[:,-2]
confirmed_ref_val =             max(confirmed_ref_val_curr)-max(confirmed_ref_val_previous)
#print(confirmed_ref_val)

recovered_ref_val_curr =        recovered_df.iloc[:,-1]
recovered_ref_val_previous =    recovered_df.iloc[:,-2]
recovered_ref_val =             max(recovered_ref_val_curr)-max(recovered_ref_val_previous)
#print(recovered_ref_val)

death_ref_val_curr =            death_df.iloc[:,-1]
death_ref_val_previous =        death_df.iloc[:,-2]
deaths_ref_val =                max(death_ref_val_curr)-max(death_ref_val_previous)
#print(deaths_ref_val)

active_ref_val_curr = country_df['active'].astype(int).sort_values(ascending= False).values[0]
active_ref_val_previous = country_df['active'].astype(int).sort_values(ascending= False).values[1]
active_ref_val = active_ref_val_curr-active_ref_val_previous
#print(deaths_ref_val)


confirmed =    max(country_df['confirmed'])
recovered =    max(country_df['recovered'])
deaths    =    max(country_df['deaths'])
active    =    max(country_df['active'])


# print(confirmed)
# print(recovered)
# print(deaths)
# print(active)

display(HTML('<br><br><div style = "font-family:Trebuchet MS;font-size:40px;color:#778899;">Daily World Indicator Counts</div><br>'))


fig = go.Figure()

fig.add_trace(
        go.Indicator(
            mode = "number+delta",
            title = {"text": "<span style='font-family:Trebuchet MS;font-size:30px;color:#0000FF'><b>Confirmed</b></span>",'align':'center'},
            value = confirmed,
            align="center",
            delta = {'reference':confirmed_ref_val, 'relative': False, 'position' : "top",'valueformat' :".0f"},
            domain = {'row': 0, 'column': 0}
        )
)


fig.add_trace(go.Indicator(
    display(HTML('<br><br>')),
    mode = "number+delta",
    title = {"text": "<span style=';font-family:Trebuchet MS;font-size:30px;color:#7CFC00'><b>Recovered</b></span>",'align':'center'},
    value = recovered,
    align="center",
    delta = {'reference':recovered_ref_val, 'relative': False,'position' : "top",'valueformat' :".0f"},
    domain = {'row': 1, 'column': 0}
))


fig.add_trace(go.Indicator(
    mode = "number+delta",
    title = {"text": "<span style=';font-family:Trebuchet MS;font-size:30px;color:#ff33cc'><b>Active</b></span>",'align':'center'},
    value = active,
    align="center",
    delta = {'reference': active_ref_val, 'relative': False,'position' : "top",'valueformat' :".0f"},
    domain = {'row': 0, 'column': 1}
))
  

fig.add_trace(go.Indicator(
    mode = "number+delta",
    title = {"text": "<span style=';font-family:Trebuchet MS;font-size:30px;color:#FF0000'><b>Death</b></span>",'align':'center'},
    value = deaths,
    align="center",
    delta = {'reference': deaths_ref_val, 'relative': False,'position' : "top",'valueformat' :".0f"},
    domain = {'row': 1, 'column': 1}
    
))
    

fig.update_layout(
    grid = {'rows': 2, 'columns': 2, 'pattern': "coupled"},
    template = {'data' : {'indicator': [{
        'delta' : {'reference': 90}}]
                         }})




In [50]:
ipywLayout = widgets.Layout(border='solid 2px green')


def bubble_chart(Scroll):

        figBubble = px.scatter(
                                sorted_country_df.head(Scroll),
                                x = 'country', 
                                y ='confirmed',
                                size= 'confirmed',
                                color = 'country', 
                                hover_name='country' ,
                                size_max=60
                              )
        figBubble.update_layout(

                                title="Top Worst Hit Countries",
                                yaxis_title="Count of Confirmed Recoverd & Death Cases",
                                xaxis_title="Countries",
                                font=dict(family="Trebuchet MS",size=16,color="#778899"),
                                xaxis_showgrid=True,
                                yaxis_showgrid=True,
                                #paper_bgcolor='rgba(0,0,0,0)',
                                plot_bgcolor='rgba(0,0,0,0)',
                                #margin=dict(l=100, r=20, t=0, b=20)

                            )

        #figBubble.update_layout(autosize=False,width=700,height=600),
        #figBubble.update_xaxes(showgrid=True, gridwidth=1, gridcolor='#DCDCDC')
        #figBubble.update_yaxes(showgrid=True, gridwidth=1, gridcolor='#DCDCDC')
        figBubble.update_xaxes(zeroline=True, zerolinewidth=2, zerolinecolor='#DCDCDC')
        figBubble.update_yaxes(zeroline=True, zerolinewidth=2, zerolinecolor='#DCDCDC')

        figBubble.show();

display(HTML('<br><br><div style = "font-family:Trebuchet MS;font-size:30px;color:#778899;">Scroll to Compare World Top Worst Hit Countries</div><br>'))

interact(bubble_chart, Scroll=10);

interactive(children=(IntSlider(value=10, description='Scroll', max=30, min=-10), Output()), _dom_classes=('wi…

In [73]:

menu = np.array(country_df['country'].values.tolist()) 
menu = np.unique(menu)

CountryBarWidget = widgets.Dropdown(options=menu,
    value='US',
    description='<p style = "font-family:Trebuchet MS;font-size:15px;font-style:bold;font-color:#778899;"><b>Countries</b></p> :</br></br>',
    disabled=False
)

def countryBar(country):
    country = country_df.loc[country_df['country'] == country]
   
    figBar = go.Figure(
        data=[
            go.Bar(
                name="Confirmed",
                x= country['country'],
                y= country['confirmed'],
                marker=dict(color='#0000FF')
               
                
            ),
            go.Bar(
                name="Deaths",
                x=country['country'],
                y=country['deaths'],
                marker=dict(color='#FF0000')
                
            ),
            go.Bar(
                name="Recovered",
                x=country['country'],
                y=country['recovered'],
                marker=dict(color='#7CFC00')
                
            ),
        
        ],
        layout=go.Layout(
            title="Confirmed Recoverd And Death Cases over the world",
            yaxis_title="Count of Confirmed Recoverd & Death Cases",
            xaxis_title="Country",
            font=dict(family="Trebuchet MS",size=16,color="#778899"),
            xaxis_showgrid=True,
            yaxis_showgrid=True,
            width=700,
            height=600,
            paper_bgcolor='rgba(0,0,0,0)',
            plot_bgcolor='rgba(0,0,0,0)'
        )
        
    )
    #figBar.update_xaxes(showgrid=True, gridwidth=1, gridcolor='#DCDCDC')
    #figBar.update_yaxes(showgrid=True, gridwidth=1, gridcolor='#DCDCDC')
    figBar.update_xaxes(zeroline=True, zerolinewidth=2, zerolinecolor='#DCDCDC')
    figBar.update_yaxes(zeroline=True, zerolinewidth=2, zerolinecolor='#DCDCDC'),
    fig.update_yaxes(automargin=True)
    
    display(HTML('<br><br><br><br>'))
    figBar.show();
    display(HTML('<br><br><br><br>'))

display(HTML('<br><br><div style = "font-family:Trebuchet MS;font-size:24px;color:#778899;">Select the Country from Drop Down</div><br>'))    
widgets.interact(countryBar,country = CountryBarWidget);

interactive(children=(Dropdown(description='<p style = "font-family:Trebuchet MS;font-size:15px;font-style:bol…

In [74]:

def recoverdDateWise(country):
    
    recovered = recovered_df.loc[recovered_df['country'] == country]
    recovered.drop(['state','lat','long'], axis =1, inplace =True)
    recovered.drop(['country'], axis =1, inplace =True)
    
    for i in recovered.values:
            y_recovered = i
            #print('length----recovered',max(y_recovered))
     
    
    death = death_df.loc[death_df['country'] == country]
    death.drop(['state','lat','long'], axis =1, inplace =True)
    death.drop(['country'], axis =1, inplace =True)
    
    for j in death.values:
        y_death = j
        #print('length----death',max(y_death))
            
    confirmed = confirmed_df.loc[confirmed_df['country'] == country]
    confirmed.drop(['state','lat','long'], axis =1, inplace =True)
    confirmed.drop(['country'], axis =1, inplace =True)
    
    for k in confirmed.values:
            y_confirmed= k
            #print('length----confirmed',max(y_confirmed))
            
            
            
   # max_range = 0 
   # print('value of max_range----',max_range)
    
    if max(y_recovered) >= max(y_death) and max(y_recovered) >= max(y_confirmed):
        max_range = max(y_recovered)
        #print('I am in recovered',max_range)
    
    elif max(y_death) >= max(y_confirmed) and max(y_death) >= max(y_recovered):
        max_range = max(y_death)
        #print('I am in death',max_range)
    
    elif max(y_confirmed) >= (y_death) and max(y_confirmed) >= max(y_recovered):
        max_range = max(y_confirmed)
        #print('I am in confirmed',max_range)

    figLine = go.Figure(
        data=[
            
             go.Line(
                name="Confirmed",
                x= confirmed.columns,
                y= y_confirmed,
                marker=dict(color='#0000FF')
               
                
            ),
            
            go.Bar(
                name="Recovered",
                x=recovered.columns,
                y=y_recovered,
                marker=dict(color='#7CFC00')
                
            ),
            
            go.Bar(
                name="Deaths",
                x=death.columns,
                y=y_death,
                marker=dict(color='#FF0000')
                
            ),
            

    
        ],
    
    
    layout=go.Layout(
            showlegend=True,
            title="Date wise Count of Confirmed Recoverd and Death Cases in "+country,
            yaxis_title="Confirmed Recoverd and Death cases Count",
            xaxis_title="Dates",
            font=dict(family="Trebuchet MS",size=16,color="#778899"),
            xaxis_showgrid=True,
            yaxis_showgrid=True,
            #paper_bgcolor='rgba(0,0,0,0)',
            plot_bgcolor='rgba(0,0,0,0)'
            
        
        )
    
    )
            
    figLine.update_xaxes(rangeslider_visible=True)
    
    figLine.update_yaxes(range=[0, max_range])
    figLine.update_layout(autosize=False,width=975,height=680),
    
    
    #figLine.update_xaxes(showgrid=True, gridwidth=1, gridcolor='#DCDCDC')
    #figLine.update_yaxes(showgrid=True, gridwidth=1, gridcolor='#DCDCDC')
    figLine.update_xaxes(zeroline=True, zerolinewidth=2, zerolinecolor='#DCDCDC')
    figLine.update_yaxes(zeroline=True, zerolinewidth=2, zerolinecolor='#DCDCDC')
    
    display(HTML('<br><br><br><br>'))
    figLine.show();

    
menu = np.array(country_df['country'].values.tolist()) 
menu = np.unique(menu)


RecoveredDropdown = widgets.Dropdown(options=menu,
    value='US',
    description='<p style = "font-family:Trebuchet MS;font-size:15px;font-style:bold;font-color:#778899;"><b>Countries</b></p> :</br></br>',
    disabled=False
)

display(HTML('<br><br><div style = "font-family:Trebuchet MS;font-size:24px;color:#778899;">Select the Country from Drop Down</div><br>'))    
widgets.interact(recoverdDateWise,country = RecoveredDropdown);

interactive(children=(Dropdown(description='<p style = "font-family:Trebuchet MS;font-size:15px;font-style:bol…

In [75]:

def recoverdDateWise(country):
    recovered = recovered_df.loc[recovered_df['country'] == country]
    recovered.drop(['state','lat','long'], axis =1, inplace =True)
    recovered.drop(['country'], axis =1, inplace =True)
    
    
    death = death_df.loc[death_df['country'] == country]
    death.drop(['state','lat','long'], axis =1, inplace =True)
    death.drop(['country'], axis =1, inplace =True)
    
    
    confirmed = confirmed_df.loc[confirmed_df['country'] == country]
    confirmed.drop(['state','lat','long'], axis =1, inplace =True)
    confirmed.drop(['country'], axis =1, inplace =True)
    
    
    
    
    for i in recovered.values:
            y= i
            #print(y)
    
    
    figLine = go.Figure(
        data=[
            go.Scatter(
                name="Recovered",
                x= recovered.columns,
                y= y,
                line={'color': '#7CFC00'},
                
            ),
    ],
    layout=go.Layout(
            showlegend=True,
            title="Count of Recoverd Cases in "+country,
            yaxis_title="Recovered cases Count",
            xaxis_title="Dates",
            font=dict(family="Trebuchet MS",size=16,color="#778899"),
            xaxis_showgrid=True,
            yaxis_showgrid=True,
            #paper_bgcolor='rgba(0,0,0,0)',
            plot_bgcolor='rgba(0,0,0,0)'
            
        
        )
    
    )
    figLine.update_xaxes(rangeslider_visible=True),
    figLine.update_yaxes(range=[0, y])
    figLine.update_layout(autosize=False,width=975,height=680)
    
    
    #figLine.update_xaxes(showgrid=True, gridwidth=1, gridcolor='#DCDCDC')
    #figLine.update_yaxes(showgrid=True, gridwidth=1, gridcolor='#DCDCDC')
    figLine.update_xaxes(zeroline=True, zerolinewidth=2, zerolinecolor='#DCDCDC')
    figLine.update_yaxes(zeroline=True, zerolinewidth=2, zerolinecolor='#DCDCDC')
    display(HTML('<br><br><br><br>'))
    figLine.show();

    
    
menu = np.array(recovered_df['country'].values.tolist()) 
menu = np.unique(menu)

RecoveredDropdown = widgets.Dropdown(options=menu,
    value='US',
    description='<p style = "font-family:Trebuchet MS;font-size:15px;font-style:bold;font-color:#778899;"><b>Countries</b></p> :</br></br>',
    disabled=False
)

display(HTML('<br><br><div style = "font-family:Trebuchet MS;font-size:24px;color:#778899;">Select the Country from Drop Down</div><br>'))    
widgets.interact(recoverdDateWise,country = RecoveredDropdown);

interactive(children=(Dropdown(description='<p style = "font-family:Trebuchet MS;font-size:15px;font-style:bol…

In [76]:

def DeathDateWise(country):
    death = death_df.loc[death_df['country'] == country]
    death.drop(['state','lat','long'], axis =1, inplace =True)
    death.drop(['country'], axis =1, inplace =True)
    
    for i in death.values:
            y= i
            #print(y)

    figLine = go.Figure(
        data=[
        
            go.Line(
                name="Deaths",
                x= death.columns,
                y= y,
                line={'color': '#FF0000'}

            ),
        
        ],
        layout=go.Layout(
            showlegend=True,
            title="Count of Deaths Cases in "+country,
            yaxis_title="Deaths cases Count",
            xaxis_title="Dates",
            font=dict(family="Trebuchet MS",size=16,color="#778899"),
            xaxis_showgrid=True,
            yaxis_showgrid=True,
            #paper_bgcolor='rgba(0,0,0,0)',
            plot_bgcolor='rgba(0,0,0,0)'
        )
    )
    figLine.update_xaxes(rangeslider_visible=True)
    figLine.update_yaxes(range=[0, y])
    figLine.update_layout(autosize=False,width=975,height=680)
    
    
    #figLine.update_xaxes(showgrid=True, gridwidth=1, gridcolor='#DCDCDC')
    #figLine.update_yaxes(showgrid=True, gridwidth=1, gridcolor='#DCDCDC')
    figLine.update_xaxes(zeroline=True, zerolinewidth=2, zerolinecolor='#DCDCDC')
    figLine.update_yaxes(zeroline=True, zerolinewidth=2, zerolinecolor='#DCDCDC')
    display(HTML('<br><br><br><br>'))
    figLine.show();

menu = np.array(death_df['country'].values.tolist()) 
menu = np.unique(menu)
 

DeathDropdown = widgets.Dropdown(options=menu,
    value='US',
    description='<p style = "font-family:Trebuchet MS;font-size:15px;font-style:bold;font-color:#778899;"><b>Countries</b></p> :</br></br>',

)

display(HTML('<br><br><div style = "font-family:Trebuchet MS;font-size:24px;color:#778899;">Select the Country from Drop Down</div><br>'))    
widgets.interact(DeathDateWise,country = DeathDropdown);

interactive(children=(Dropdown(description='<p style = "font-family:Trebuchet MS;font-size:15px;font-style:bol…

In [77]:

def ConfirmedDateWiseCases(country):
    
    confirmed = confirmed_df.loc[confirmed_df['country'] == country]
    confirmed.drop(['state','lat','long'], axis =1, inplace =True)
    confirmed.drop(['country'], axis =1, inplace =True)
    
    for i in confirmed.values:
            y= i
            #print(y)

    figLine = go.Figure(
        data=[
        
            go.Line(
                name="Confirmed",
                x= confirmed.columns,
                y= y,
                line={'color': '#0000FF'}

            
            ),

        ],
        layout=go.Layout(
            showlegend=True,
            title="Count of Confirmed Cases over the world " +country,
            yaxis_title="Confirmed cases Count",
            xaxis_title="Dates",
            font=dict(family="Trebuchet MS",size=16,color="#778899"),
            xaxis_showgrid=True,
            yaxis_showgrid=True,
            #paper_bgcolor='rgba(0,0,0,0)',
            plot_bgcolor='rgba(0,0,0,0)'
        )
    )
    figLine.update_xaxes(rangeslider_visible=True)
    figLine.update_yaxes(range=[0, y])
    figLine.update_layout(autosize=False,width=975,height=680)
    
    
    #figLine.update_xaxes(showgrid=True, gridwidth=1, gridcolor='#DCDCDC')
    #fbbigLine.update_yaxes(showgrid=True, gridwidth=1, gridcolor='#DCDCDC')
    figLine.update_xaxes(zeroline=True, zerolinewidth=2, zerolinecolor='#DCDCDC')
    figLine.update_yaxes(zeroline=True, zerolinewidth=2, zerolinecolor='#DCDCDC')
    
    display(HTML('<br><br><br><br>'))
    figLine.show();

    
menu = np.array(confirmed_df['country'].values.tolist()) 
menu = np.unique(menu)

ConfirmedDropdown = widgets.Dropdown(options=menu,
    value='US',
    description='<p style = "font-family:Trebuchet MS;font-size:15px;font-style:bold;font-color:#778899;"><b>Countries</b></p> :</br></br>',

)

display(HTML('<br><br><div style = "font-family:Trebuchet MS;font-size:24px;color:#778899;">Select the Country from Drop Down</div><br>'))    
widgets.interact(ConfirmedDateWiseCases,country = ConfirmedDropdown);

interactive(children=(Dropdown(description='<p style = "font-family:Trebuchet MS;font-size:15px;font-style:bol…

In [78]:
display(HTML('<br><br><div style = "font-family:Trebuchet MS;font-size:24px;color:#778899;">Hover Over World Map</div><br>'))    

world_map = folium.Map(location=[19.0760,72.8777], tiles="cartodbpositron", zoom_start=2, max_zoom = 6, min_zoom = 2)


for i in range(0,len(confirmed_df)):
    folium.Circle(
        location=[confirmed_df.iloc[i]['lat'], confirmed_df.iloc[i]['long']],
        fill=True,
        radius=(int((np.log(confirmed_df.iloc[i,-1]+1.00001)))+0.2)*50000,
        color='	 #ffb3d9',
        fill_color='indigo',
        tooltip = "<div style='margin: 0; background-color: #cccccc; color: #000000;'>"+
                    "<h4 style='text-align:center;font-weight: bold'>"+confirmed_df.iloc[i]['country'] + "</h4>"
                    "<hr style='margin:10px;color: #4d4d4d;'>"+
                    "<ul style='color: #000000;font-weight: bold;list-style-type:circle;align-item:left;padding-left:20px;padding-right:20px'>"+
                        "<li>Confirmed: "+str(confirmed_df.iloc[i,-1])+"</li>"+
                        "<li>Deaths:   "+str(death_df.iloc[i,-1])+"</li>"+
                        "<li>Death Rate: "+ str(np.round(death_df.iloc[i,-1]/(confirmed_df.iloc[i,-1]+1.00001)*100,2))+ "</li>"+
                    "</ul></div>",
        ).add_to(world_map)

world_map



In [68]:
display(HTML(
    
            '<html>'
                '<head>'
                         '<style>'
                                'a:link {color: #0000ff;}'              # unvisited link
                                'a:visited {color: #e600ac;}'           # visited link 
                                'a:hover {color: #2eb82e;}'            # mouse over link
                                'a:active {color: #00ff00;}'          # selected link 
                       '</style>'
                    '</head>'
                '<body>'

                    '<div style = "Lucida Console, Courier, monospace;font-size:20px;color:#778899;line-height: 1.7;text-align: justify;">'+

                        '<h2>Notebook covers:</h2>'
                         
                            '1. What is COVID-19?<br>'+
                            '2. Data loading from John Hopkins CSSE data repository?<br>'+
                            '3. Data Cleaning and Preparation?<br>'+
                            '4. Visualising N number of worst hit countries using plotly scatter plot.?<br>'+
                            '5. Plotting confirmed and death cases for the requested country.?<br>'+
                            '6. Plotting all cases on world map using Folium?</li><br><br>'+
                        



                        '<h2>Symptoms:</h2>'+
                        'People may be sick with the virus for 1 to 14 days before developing symptoms. The most common symptoms of coronavirus disease (COVID-19) are fever, tiredness, and dry cough. Most people (about 80%)  recover from the disease without needing special treatment.<br>'+
                          
                            '1. cough.<br>'+
                            '2. fever.<br>'+
                            '3. tiredness.<br>'+
                            '4. difficulty in breathing (severe cases).<br>'+
                        


                        '<h2>More Info on COVID-19:</h2>'+
                        '<a style="text-decoration:none" href = https://www.who.int/health-topics/coronavirus>  https://www.who.int/health-topics/coronavirus </a><br>'+
                        '<a style="text-decoration:none" href = https://www.who.int/emergencies/diseases/novel-coronavirus-2019>  https://www.who.int/emergencies/diseases/novel-coronavirus-2019  </a><br>'+
                        '<a style="text-decoration:none" href = https://www.nature.com/articles/s41597-020-0448-0>  https://www.nature.com/articles/s41597-020-0448-0  </a>'
                '</div>'
          '</body>'
        '</html>'
  

        ))
    


In [69]:
%%HTML

    
<div style = "Lucida Console, Courier, monospace;font-size:20px;color:#778899;">
<marquee direction="left" height="100px" width="90%" scrollamount="15"><h2>Disclaimer: This is not a official website. For more Details Visit to the Given Link  !!</h2></marquee>
</div>
    

